In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from astropy.table import Table
roots_tfm = '/home/julien/Documents/Etudes/Astrofisica/Master/TFM/Data'

In [ ]:
#df = dd.read_csv(roots_tfm + '/Galex/GUVCat_AIS_FOV055_glat50_00N__55_00N.csv', \
#                 names=['objid','ra','dec','tilenum','nuv_flux','nuv_fluxerr', \
#                         'fuv_flux','fuv_fluxerr','nuv_poserr'],header=0)
df = pd.read_csv(roots_tfm + '/Galex/GUVCat_AIS_FOV055_glat50_00N__55_00N_lite.csv')
#df = pd.read_csv(roots_tfm + '/Galex/GUVCat_AIS_FOV055_glat05_00N__10_00N_lite.csv')

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df = df.rename(columns={'ra':'RA','dec':'DEC','nuv_poserr':'POSERR','objid':'SRCNUM',\
                       'nuv_flux':'NUV_FLUX', 'nuv_fluxerr' : 'NUV_FLUX_ERR', \
                         'fuv_flux': 'FUV_FLUX','fuv_fluxerr' : 'FUV_FLUX_ERR'})

In [ ]:
df.head()

In [ ]:
df = df.astype({'SRCNUM': np.int64, 'NUV_FLUX': np.float32, 'FUV_FLUX': np.float32, \
                'NUV_FLUX_ERR': np.float32, 'FUV_FLUX_ERR': np.float32, 'RA': np.float64, \
                'DEC': np.float64, 'POSERR': np.float32, 'tilenum' : np.int32})

In [ ]:
def create_source_ra_dec_file(df):

    # Treatment of points on the right ascension edge
    df_source_test = df.groupby('SRCNUM').agg({'RA' : 'std'})
    ind_src_ra_err = df_source_test[df_source_test['RA']>10].index
    if len(ind_src_ra_err)>0:
        ind_ra_err = (df['SRCNUM'].isin(ind_src_ra_err.values)) & (df['RA']>180)
        df.loc[ind_ra_err,'RA']  = df.loc[ind_ra_err,'RA'] - 360   

    # Computation of the weighted column and creation of the utils column to aggregate
    df['RA_weighted'] = df['RA']/df['POSERR']
    df['DEC_weighted'] = df['DEC']/df['POSERR']
    df['1/POSERR'] = 1/df['POSERR'].copy()
    df['COUNT'] = df['SRCNUM'].copy()

    # Aggregation
    df_source = df.groupby('SRCNUM').agg({'RA_weighted' : 'sum', 'DEC_weighted' : 'sum', 
                                         '1/POSERR': 'sum', 'POSERR': 'mean', 'COUNT': 'count'})
    
    # Computation of the weighted mean    
    df_source['DEC_weighted'] = df_source['DEC_weighted']/df_source['1/POSERR']
    df_source.loc[:, 'RA_weighted'] = df_source['RA_weighted']/df_source['1/POSERR']

    # Sanity check
    test = 0
    if df_source.loc[:, 'RA_weighted'].min() < 0:
        test = -1
        print('Sanity check start: Some points have a negative right ascension before correction')
    
    # Points on the edge are relocated in the right place if necessary (negative => +360°)
    df_source.loc[df_source.loc[:, 'RA_weighted']<0, 'RA_weighted'] = df_source.loc[df_source.loc[:, 'RA_weighted']<0, 'RA_weighted'] + 360

    # Sanity check end
    if test == -1:
        if df_source.loc[:, 'RA_weighted'].min() > 0:
            print('Sanity check ok: No negative right ascension remaining after correction')
        else:
            print('Error: Negative right ascension remaining!!')
    
    # Final cleaning
    df_source = df_source.rename(columns={'RA_weighted':'RA','DEC_weighted': 'DEC'}).drop('1/POSERR',axis=1)

    # File saving

    filepath = roots_tfm + '/Galex/Galex_ra_dec'
    t1 = Table.from_pandas(df_source,index=True)
    t1.write(filepath.split(".")[0] +'_per_src.fits',overwrite='True')

In [ ]:
df_radec = df[['RA','DEC','POSERR','SRCNUM']]

In [ ]:
#create_source_ra_dec_file(df_radec)

In [ ]:
df_radec.shape

In [ ]:
df_entries = df[['SRCNUM','NUV_FLUX','NUV_FLUX_ERR','FUV_FLUX','FUV_FLUX_ERR','tilenum']]
#df_entries.to_csv(roots_tfm + '/Galex/Entries_Galex_glat50_00N_55_00N_csv')

In [ ]:
import dask.dataframe as dd

In [ ]:
dat = dd.read_parquet(roots_tfm + '/Swift-UVOT/Entries_UVOT_parquet')
df_UVOT_match = pd.read_csv(roots_tfm + '/Galex/Galex_match_with_UVOT')
#df_UVOT_match = pd.read_csv(roots_tfm + '/Galex/Galex2_match_with_UVOT')
dat_UVOT = dat[dat['SRCNUM'].isin(df_UVOT_match['SRCNUM_2'])].compute()

In [ ]:
df_UVOT_match.head()

In [ ]:
#dat = dd.read_parquet(roots_tfm + '/XMM-Newton/Entries_OM_parquet')
#df_OM_match = pd.read_csv(roots_tfm + '/Galex/Galex_match_with_OM')
#dat_OM = dat[dat['SRCNUM'].isin(df_OM_match['SRCNUM_2'])].compute()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [ ]:
dat_UVOT.shape

In [ ]:
for flux in ['UVW2','UVM2','UVW1']: 
  
    flux_name = flux + '_FLUX'
    flux_err = flux + '_FLUX_ERR'
    flag_extended = flux + '_EXTENDED'
    flux_quality = flux + '_QUALITY_FLAG'

    # Dataframe UVOT for this band
    df_flux_uvot = dat_UVOT[['SRCNUM',flux_name,flux_err,flag_extended, flux_quality]].dropna(subset=[flux_name]).reset_index() 
    ind_minflux = df_flux_uvot.groupby('SRCNUM').agg({flux_err : 'idxmin'})[flux_err]
    df_flux_uvot = df_flux_uvot.loc[ind_minflux,:].rename(columns={flux_name: flux_name + '_UVOT', \
                                                                   flux_err: flux_err + '_UVOT', \
                                                                   flag_extended: flag_extended  + '_UVOT',\
                                                                   flux_quality: flux_quality + '_UVOT',\
                                                                   'SRCNUM': 'SRCNUM_2'})

    # Data frame OM for this band
    #df_flux_om = dat_OM[['SRCNUM',flux_name,flux_err]].dropna(subset=[flux_name]).reset_index()        
    #ind_minflux = df_flux_om.groupby('SRCNUM').agg({flux_err : 'idxmin'})[flux_err]
    #df_flux_om = df_flux_om.loc[ind_minflux,:].rename(columns={flux_name: flux_name + '_OM',\
    #                                                           flux_err: flux_err + '_OM', \
    #                                                           'SRCNUM': 'SRCNUM_2'})

    for flux_galex in ['NUV','FUV']:
        
        flux_galex_name = flux_galex + '_FLUX'
        flux_galex_err = flux_galex + '_FLUX_ERR'

         # Data frame Galex for this band
        df_flux_galex = df_entries[['SRCNUM',flux_galex_name,flux_galex_err]].dropna(subset=[flux_galex_name])
        df_flux_galex=df_flux_galex.reset_index().rename(columns={'SRCNUM': 'SRCNUM_1'})

        # Merge with UVOT results
        df_merge = df_UVOT_match.merge(df_flux_galex, on='SRCNUM_1').merge(df_flux_uvot, on='SRCNUM_2')

        # Plot figure
        fig, ax =plt.subplots()
        
        ind_extended = (df_merge[flag_extended+ '_UVOT'] == "False")
        ind_quality = (df_merge[flux_quality + '_UVOT'] == 0)
        ind_nan = (df_merge[flux_galex_name] == -999)
        
        ind1 = ind_extended & ind_quality & ~ind_nan
        ax.scatter(np.log10(df_merge.loc[ind1,flux_name + '_UVOT']), np.log10(df_merge.loc[ind1,flux_galex_name]), c='m',marker='x', \
                   label='Ext. src')
        z = np.polyfit(np.log10(df_merge.loc[ind1,flux_name + '_UVOT']),np.log10(df_merge.loc[ind1,flux_galex_name]), deg=1)
        p = np.poly1d(z)
        xp = np.linspace(np.log10(df_merge.loc[ind1,flux_name + '_UVOT']).min(), np.log10(df_merge.loc[ind1,flux_name + '_UVOT']).max(), 100)
        ax.plot(xp,p(xp), color='r', linestyle='dashed', label= "Ext. src log(Galex) = " + 
                  "{:.2E}".format(p[1]) + " log(UVOT) + " + "{:.2E}".format(p[0]))

        ind2 = ~ind_extended & ind_quality & ~ind_nan
        ax.scatter(np.log10(df_merge.loc[ind2,flux_name + '_UVOT']), np.log10(df_merge.loc[ind2,flux_galex_name]), c='c',marker='x', \
                   label='Not ext. src')
        z = np.polyfit(np.log10(df_merge.loc[ind2,flux_name + '_UVOT']),np.log10(df_merge.loc[ind2,flux_galex_name]), deg=1)
        p = np.poly1d(z)
        xp = np.linspace(np.log10(df_merge.loc[ind2,flux_name + '_UVOT']).min(), np.log10(df_merge.loc[ind2,flux_name + '_UVOT']).max(), 100)
        ax.plot(xp,p(xp), color='b', linestyle='dashed', label= "Not ext. src log(Galex) = " + 
                  "{:.2E}".format(p[1]) + " log(UVOT) + " + "{:.2E}".format(p[0]))
        ax.set_xlabel("UVOT - " + flux + ' Flux [' + "$erg.s^{-1}.cm^{2}.A^{-1}]$")
        ax.set_ylabel("Galex - " + flux_galex + ' Flux  [' + "$\mu$" +"Jy]")
        ax.legend(loc='upper right',prop={'size': 8},title= flux + " band vs " + flux_galex + " band")
        fig.savefig(roots_tfm + '/Calibration/UVOT_' + flux + '_vs_Galex_ ' + flux_galex + "_glat05_00N__10_00N.png",dpi=1200)  

In [ ]:
df_merge.head()